# Generating Files for Joins

Using SX generate parquet files for testing with join infrastructure. Also, using new SX frontend.

Constants:

In [1]:
# A Z to ee sample - Release 21
ds_name = r"mc16_13TeV:mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.DAOD_PHYS.e3601_e5984_s3126_r10201_r10210_p5313"

What we'll need for imports

In [2]:
from servicex import ServiceXClient, RucioDatasetIdentifier, ResultFormat, DatasetGroup
from func_adl_servicex_xaodr22.event_collection import Event

# import logging
# logging.basicConfig(level=logging.DEBUG)

The base dataset creation:

In [3]:
sx = ServiceXClient(backend="test4")
did = RucioDatasetIdentifier(ds_name, num_files=10)
ds_raw = sx.func_adl_dataset(
    did, codegen="atlasr21", title="Zee", result_format=ResultFormat.parquet, item_type=Event
)

from func_adl_servicex_xaodr22 import calib_tools
# ds = calib_tools.apply_calibration(ds_raw, "PHYS") <this is what we should have>
ds = calib_tools.query_update(ds_raw, calib_tools.default_config("PHYSLITE"))

# Central Electron $p_T$

We'll grab all the central electron $p_T$ that are more than 25 GeV.

In [4]:
good_ele = ds.Select(
    lambda e: {
        "run": e.EventInfo("EventInfo").runNumber(),
        "event": e.EventInfo("EventInfo").eventNumber(),
        "good_ele": e.Electrons("Electrons").Where(lambda e: (e.pt() / 1000 > 25.0) and (abs(e.eta()) < 2.5)
        ),
    }
)

First, just the $p_T$, and then the $\eta$:

In [5]:
electron_pt = good_ele.Select(lambda e: {
    "run": e.run,
    "event": e.event,
    "pt": e.good_ele.Select(lambda ele: ele.pt()/1000.0),
})

In [6]:
r = electron_pt.as_signed_urls()
print(f"number of URLs: {len(r.signed_url_list)}")

Output()

ServiceX Transform fff976d6-6c79-4d37-a1c9-4c9ccc73ac70

Transforms completed successfully

number of URLs: 10


In [7]:
r = electron_pt.as_files()
print(len(r.file_list))

Output()

10


In [8]:
electron_etaphi = good_ele.Select(lambda e: {
    "run": e.run,
    "event": e.event,
    "eta": e.good_ele.Select(lambda ele: ele.eta()),
    "phi": e.good_ele.Select(lambda ele: ele.phi()),
})

And the number of calorimeter clusters, along with $p_T$ for matching

In [9]:
electron_calo = good_ele.Select(lambda e: {
    "run": e.run,
    "event": e.event,
    "pt": e.good_ele.Select(lambda ele: ele.pt()/1000.0),
    "n_calo": e.good_ele.Select(lambda ele: ele.nCaloClusters()),
})

## Another slice of electrons

Grab $\eta$, $\phi$, and $p_T$ for electrons between 20 and 25 GeV, but central.

In [10]:
medium_ele = ds.Select(
    lambda e: {
        "run": e.EventInfo("EventInfo").runNumber(),
        "event": e.EventInfo("EventInfo").eventNumber(),
        "good_ele": e.Electrons("Electrons").Where(lambda e: (e.pt() / 1000 > 20.0) and (e.pt() / 1000 <= 25.0) and (abs(e.eta()) < 2.5)
        ),
    }
)

In [11]:
medium_ele_ptetaphi = medium_ele.Select(lambda e: {
    "run": e.run,
    "event": e.event,
    "pt": e.good_ele.Select(lambda ele: ele.pt()/1000.0),
    "eta": e.good_ele.Select(lambda ele: ele.eta()),
    "phi": e.good_ele.Select(lambda ele: ele.phi()),
    })

## The Missing $E_T$

This is an event level variable, so there is only one of these.

There is currently a bug, so this isn't working yet! This bug is now fixed in the main branch, but has not been released yet.

In [12]:
# missing_et = ds.Select(lambda e: {
#     "run": e.EventInfo("EventInfo").runNumber(),
#     "event": e.EventInfo("EventInfo").eventNumber(),
#     "missing_et": e.MissingET().met()/1000.0,
# })

## Number of interactions per crossing

This is another thing where there is a single item per event, so it can take the place of missing $E_T$ until that is fixed.

In [13]:
crossings = ds.Select(lambda e: {
    "run": e.EventInfo("EventInfo").runNumber(),
    "event": e.EventInfo("EventInfo").eventNumber(),
    "interactions": e.EventInfo("EventInfo").actualInteractionsPerCrossing(),
})

## Fetch it all

Now that we have build all the queries, lets fetch them!

In [14]:
everything = {
    "electron_pt": electron_pt,
    "electron_etaphi": electron_etaphi,
    "medium_ele_ptetaphi": medium_ele_ptetaphi,
    # "missing_et": missing_et,
    "interactions": crossings,
    "n_calo": electron_calo,
}

Use the dataset group to get them all to show up, and then rebuild the results into a dictionary so we can easily print out the information we need.

In [15]:
d = DatasetGroup([v for v in everything.values()])

r_list = d.as_signed_urls()
everything_results = {
    k: v
    for k, v in zip(everything.keys(), r_list)
}
print(len(everything_results))

Output()

Returning results from cache

ServiceX Transform 1c62c098-2349-4e6d-aa33-156c3f762f72

ServiceX Transform 5c604d77-8def-4f05-92c6-86e27a3495d7

ServiceX Transform 785c44b6-47ce-4e85-9568-10d28691448d

...

Transforms completed successfully

Transforms completed successfully

Transforms completed successfully

Transforms completed successfully

5


In [16]:
from rich.table import Table

t = Table(title="Zee")
t.add_column("Name")
t.add_column("Request ID")
t.add_column("# Files")

for k,v in everything_results.items():
    t.add_row(k, v.request_id, str(len(v.signed_url_list)))

from rich.console import Console
console = Console()
console.print(t)

                                  Zee                                   
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Name                ┃ Request ID                           ┃ # Files ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ electron_pt         │ fff976d6-6c79-4d37-a1c9-4c9ccc73ac70 │ 10      │
│ electron_etaphi     │ 5c604d77-8def-4f05-92c6-86e27a3495d7 │ 10      │
│ medium_ele_ptetaphi │ f3f14d13-c1eb-4d99-892b-e5f0fe8f0801 │ 10      │
│ interactions        │ 1c62c098-2349-4e6d-aa33-156c3f762f72 │ 10      │
│ n_calo              │ 785c44b6-47ce-4e85-9568-10d28691448d │ 10      │
└─────────────────────┴──────────────────────────────────────┴─────────┘